### TDA 2016, January 21st, Leuven

# &nbsp;

# TensorOperations.jl:
## Convenient tensor operations with Julia
### (and fun with metaprogramming)

# &nbsp;

### Jutho Haegeman
#### Department of Physics and Astronomy, UGent

## Overview


* **Motivation: Tensor Network Decompositions in Quantum Physics**
* **Intro to the Julia Language**
* **TensorOperations.jl**
* **Implementation of basic tensor operations (with metaprogramming)**
* **Optimization of tensor contraction order**
* **Outlook**

## Motivation: quantum many body physics
* weirdness of quantum mechanics: Schrodinger's cat
<img src="schrodinger.png" style="width: 300px;"/>

## Motivation: quantum many body physics
* quantum bit ( = qubit):

$$\vert\Psi\rangle = \alpha \vert 0\rangle + \beta \vert 1\rangle\quad\text{with}\quad\alpha,\beta\in\mathbb{C}$$

* intrinsically indeterministic:

    * $|\alpha|^2$: probability of measuring 0
    * $|\beta|^2$: probability of measuring 1

* for $N$ different qubits? 

$$\vert\Psi\rangle = \Psi_{00000}\vert 00000 \rangle + \Psi_{00001} \vert 00001\rangle + \ldots+ \Psi_{11111} \vert 11111\rangle$$

**$\Rightarrow$ storing a quantum state of $N$ qubits requires $2^N$ complex numbers: $\Psi_{i_1,i_2,\ldots,i_{N}}$**

## Motivation: quantum many body physics
* quantum state is a high-order tensor / multidimensional array:
  <img src="psi.png" style="width: 500px;"/>

* Curse of dimensionality: exponential scaling in $N$, the number of degrees of freedom (qubits, spins, atoms, ...)
  
* Realistic materials: $N$ is in the order of Avogadro's number, i.e. $O(10^{23})$
  <img src="graphene.jpg" style="width: 300px;"/>

## Motivation: tensor network decompositions
* graphical notation:
    * matrix - vector multiplication: <img src="matvec.png" style="width: 400px;"/>
    * matrix - matrix multiplication: <img src="matmat.png" style="width: 400px;"/>
* tensor network decompositions for efficient description of quantum states
  <img src="tn2.png" style="width: 600px;"/>

## Introduction to the Julia Language

  <img src="julia.png" style="width: 200px;"/>


* Selling point: dynamic high-level language with the speed of a statically-compiled language

* Key features:
    * Just-in-time compiled (using LLVM infrastructure)
    * Dynamic type system
    * Multiple dispatch:
        * define function behavior across many combinations of argument types
        * automatic generation of efficient, specialized code for different argument types
    * Good support for computational science: numerics, statistics, multidimensional arrays, ...
    * Homoiconic and powerful metaprogramming facilities

### Code generation

In [1]:
function myabs(x)
    if x < 0
        return -x
    end
    return x
end
function myabs2(x::Real)
    if x < 0
        return -x
    end
    return x
end
function myabs2(x::Unsigned)
    return x
end

myabs2 (generic function with 2 methods)

### Code generation

In [2]:
code_llvm(myabs,Tuple{Int64}) # LLVM code for 64-bit integer


define i64 @julia_myabs_21601(i64) {
top:
  %1 = icmp sgt i64 %0, -1
  br i1 %1, label %L, label %if

if:                                               ; preds = %top
  %2 = sub i64 0, %0
  ret i64 %2

L:                                                ; preds = %top
  ret i64 %0
}


In [3]:
code_llvm(myabs,Tuple{UInt64}) # LLVM code for 64-bit unsigned integer


define i64 @julia_myabs_21626(i64) {
L:
  ret i64 %0
}


In [4]:
code_llvm(myabs,Tuple{Float64}) # LLVM code for 64-bit floating point


define double @julia_myabs_21627(double) {
top:
  %1 = fcmp uge double %0, 0.000000e+00
  br i1 %1, label %L, label %if

if:                                               ; preds = %top
  %2 = fmul double %0, -1.000000e+00
  ret double %2

L:                                                ; preds = %top
  ret double %0
}


### Type inference & type stability

In [5]:
mysqrt(x) = x < 0 ? sqrt(complex(x)) : sqrt(x)
code_typed(mysqrt,Tuple{Float64})

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x], Any[Any[Any[:x,Float64,0],Any[symbol("##fy#7529"),Float64,18]],Any[],Any[],Any[]], :(begin  # In[5], line 1:
        ##fy#7529 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
        unless (Base.box)(Base.Bool,(Base.or_int)((Base.lt_float)(x::Float64,##fy#7529::Float64)::Bool,(Base.box)(Base.Bool,(Base.and_int)((Base.box)(Base.Bool,(Base.and_int)((Base.eq_float)(x::Float64,##fy#7529::Float64)::Bool,(Base.lt_float)(##fy#7529::Float64,9.223372036854776e18)::Bool)),(Base.slt_int)((Base.box)(Int64,(Base.fptosi)(Int64,##fy#7529::Float64)),0)::Bool)))) goto 0
        return (Main.sqrt)($(Expr(:new, Complex{Float64}, :(x::Float64), :((Base.box)(Float64,(Base.sitofp)(Float64,0))))))::Complex{Float64}
        0: 
        return (Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)(x::Float64))::Float64
    end::Union{Complex{Float64},Float64}))))

### Type inference & type stability

In [6]:
function summyabs(v::Vector)
    s = myabs(v[1])
    for i = 2:length(v)
        s += myabs(v[i])
    end
    return x
end
code_typed(summyabs,Tuple{Vector{Float64}})

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:v], Any[Any[Any[:v,Array{Float64,1},0],Any[:s,Float64,2],Any[symbol("#s41"),Int64,2],Any[:i,Int64,18],Any[symbol("####fy#7487#7612"),Float64,18],Any[:_var0,Float64,2],Any[symbol("####fy#7487#7613"),Float64,18],Any[:_var1,Float64,2]],Any[],Any[UnitRange{Int64},Tuple{Int64,Int64},Float64,Int64,Float64,Int64,Int64],Any[]], :(begin  # In[6], line 2:
        GenSym(2) = (Base.arrayref)(v::Array{Float64,1},1)::Float64
        ####fy#7487#7612 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
        unless (Base.box)(Base.Bool,(Base.or_int)((Base.lt_float)(GenSym(2),####fy#7487#7612::Float64)::Bool,(Base.box)(Base.Bool,(Base.and_int)((Base.box)(Base.Bool,(Base.and_int)((Base.eq_float)(GenSym(2),####fy#7487#7612::Float64)::Bool,(Base.lt_float)(####fy#7487#7612::Float64,9.223372036854776e18)::Bool)),(Base.slt_int)((Base.box)(Int64,(Base.fptosi)(Int64,####fy#7487#7612::Float64)),0)::Bool)))) goto 6
        _var0 = (Base.box)(Base.Float64,(Base.neg_fl

### Type inference & type stability

In [7]:
function summysqrt(v::Vector)
    s = mysqrt(v[1])
    for i = 2:length(v)
        s += mysqrt(v[i])
    end
    return x
end
code_typed(summysqrt,Tuple{Vector{Int64}})

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:v], Any[Any[Any[:v,Array{Int64,1},0],Any[:s,Union{Complex{Float64},Float64},2],Any[symbol("#s41"),Int64,2],Any[:i,Int64,18],Any[:_var0,Union{Complex{Float64},Float64},2],Any[:_var1,Union{Complex{Float64},Float64},2]],Any[],Any[UnitRange{Int64},Tuple{Int64,Int64},Int64,Complex{Int64},Int64,Int64,Complex{Int64},Int64,Int64],Any[]], :(begin  # In[7], line 2:
        GenSym(2) = (Base.arrayref)(v::Array{Int64,1},1)::Int64
        unless (Base.slt_int)(GenSym(2),0)::Bool goto 6
        GenSym(3) = $(Expr(:new, Complex{Int64}, GenSym(2), 0))
        _var0 = (Base.sqrt)($(Expr(:new, Complex{Float64}, :((Base.box)(Float64,(Base.sitofp)(Float64,(top(getfield))(GenSym(3),:re)::Int64))), :((Base.box)(Float64,(Base.sitofp)(Float64,(top(getfield))(GenSym(3),:im)::Int64))))))::Complex{Float64}
        goto 7
        6: 
        _var0 = (Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)((Base.box)(Float64,(Base.sitofp)(Float64,GenSym(2)))))::Float6

### Homoiconicity

In [8]:
ex=:(function summysqrt(v::Vector)
        s = mysqrt(v[1])
        for i = 2:length(v)
            s += mysqrt(v[i])
        end
        return x
    end);

In [9]:
typeof(ex)

Expr

In [10]:
println(ex.head),println(ex.args[1]),println(ex.args[2]);

function
summysqrt(v::Vector)
begin  # In[8], line 2:
    s = mysqrt(v[1]) # In[8], line 3:
    for i = 2:length(v) # In[8], line 4:
        s += mysqrt(v[i])
    end # In[8], line 6:
    return x
end


In [11]:
Meta.show_sexpr(ex)

(:function, (:call, :summysqrt, (:(::), :v, :Vector)), (:block,
    :( # In[8], line 2:),
    (:(=), :s, (:call, :mysqrt, (:ref, :v, 1))),
    :( # In[8], line 3:),
    (:for, (:(=), :i, (:(:), 2, (:call, :length, :v))), (:block,
        :( # In[8], line 4:),
        (:+=, :s, (:call, :mysqrt, (:ref, :v, :i)))
      )),
    :( # In[8], line 6:),
    (:return, :x)
  ))

### Metaprogramming

In [12]:
macro twice(ex)
    Expr(:block,ex,ex)
end

In [13]:
x=3;
@twice x+=1
x

5

## TensorOperations.jl
* general tensor operations include permutations, partial traces, contractions
    * graphical:
      <img src="tensorcontraction.png" style="width: 500px;"/>
      
    * index notation with Einstein summation convention:
      $$D_{a,b,c} = A_{a,d,e,c}\cdot B_{f,e,b,d,f}+C_{c,b,a}$$

In [14]:
n=3;
A=randn(n,n,n,n);
B=randn(n,n,n,n,n);
C=randn(n,n,n);

D=zeros(n,n,n);
for a=1:n, b=1:n, c=1:n
    D[a,b,c] += C[c,b,a]
    for d=1:n, e=1:n, f=1:n
        D[a,b,c] += A[a,d,e,c]*B[f,e,b,d,f]
    end
end

using TensorOperations
@tensor D2[a,b,c] := A[a,d,e,c]*B[f,e,b,d,f] + C[c,b,a];

@tensor D3[α,β,3] := A[α,'d',-7,3]*B[f,-7,β,'d',f] + C[3,β,α];

vecnorm(D-D2)

6.646982297870117e-15

In [15]:
function f1!(D,n,A,B,C)
    for a=1:n, b=1:n, c=1:n
        D[a,b,c] += C[c,b,a]
        for d=1:n, e=1:n, f=1:n
            D[a,b,c] += A[a,d,e,c]*B[f,e,b,d,f]
        end
    end
    return D
end
function f2!(D,n,A,B,C)
    @tensor D[a,b,c] = A[a,d,e,c]*B[f,e,b,d,f] + C[c,b,a];
    return D
end

n=30;
A=randn(n,n,n,n);
B=randn(n,n,n,n,n);
C=randn(n,n,n);
D=zeros(n,n,n);

In [16]:
@time f1!(D,n,A,B,C);
@time f2!(D,n,A,B,C);

  6.023060 seconds (19.33 k allocations: 885.976 KB)
  0.174286 seconds (176.65 k allocations: 13.658 MB)


In [17]:
@time f1!(D,n,A,B,C);
@time f2!(D,n,A,B,C);

  5.733241 seconds (4 allocations: 160 bytes)
  0.023584 seconds (158 allocations: 6.599 MB, 40.82% gc time)


### What is going on underneath?
* **Basic tensor operations** (inspired by BLAS)
    * permutations and addition: `C = β*C + α*permutation(op(A))`
    * partial trace: `C = β*C + α*partialtrace(op(A))`
    * contraction: `C = β*C + α*contract(op(A),op(B))`
    
  `op` can be idenity (doing nothing) or `conj`
    
  also available via function based syntax

## Implementation of basic tensor operations (with metaprogramming)
### 1. Permutations

In [18]:
A=randn(10,10,10,10,10,10,10,10);
B=zeros(10,10,10,10,10,10,10,10);

In [19]:
@time copy!(B,A);
@time permutedims!(B,A,[8,7,6,5,4,3,2,1]);
@time @tensor B[8,7,6,5,4,3,2,1] = A[1,2,3,4,5,6,7,8];

  0.115277 seconds (8.14 k allocations: 414.904 KB)
  2.046332 seconds (60.49 k allocations: 2.892 MB)
  1.293322 seconds (1.27 M allocations: 63.428 MB)


In [20]:
@time copy!(B,A);
@time permutedims!(B,A,[8,7,6,5,4,3,2,1]);
@time @tensor B[8,7,6,5,4,3,2,1] = A[1,2,3,4,5,6,7,8];

  0.092903 seconds (4 allocations: 160 bytes)
  1.960895 seconds (40 allocations: 1.406 KB)
  0.293874 seconds (32 allocations: 1.406 KB)


### 1. Permutations
* How to optimize permutations? Why is it slower than normal copy?
* Even for matrix transposition?
  ```julia
  transpose!(dst,src)```
  <img src="transpose.png" style="width: 400px;"/>
  Memory is linear $\Rightarrow$ `transpose` requires unfavorable memory access!

```julia
function transpose!(B::StridedMatrix,A::StridedMatrix)
    m, n = size(A)
    size(B,1) == n && size(B,2) == m || throw(DimensionMismatch("transpose"))

    if m*n<=4*transposebaselength
        @inbounds begin
            for j = 1:n
                for i = 1:m
                    B[j,i] = transpose(A[i,j])
                end
            end
        end
    else
        transposeblock!(B,A,m,n,0,0)
    end
    return B
end
function transposeblock!(B::StridedMatrix,A::StridedMatrix,m::Int,n::Int,offseti::Int,offsetj::Int)
    if m*n<=transposebaselength
        @inbounds begin
            for j = offsetj+(1:n)
                for i = offseti+(1:m)
                    B[j,i] = transpose(A[i,j])
                end
            end
        end
    elseif m>n
        newm=m>>1
        transposeblock!(B,A,newm,n,offseti,offsetj)
        transposeblock!(B,A,m-newm,n,offseti+newm,offsetj)
    else
        newn=n>>1
        transposeblock!(B,A,m,newn,offseti,offsetj)
        transposeblock!(B,A,m,n-newn,offseti,offsetj+newn)
    end
    return B
end
```

### 1. Permutations
* How to generalize to multidimensional permutations?
    1. How to write nested loops depending on the dimensionality of the array?
    2. What is the best blocking (divide and conquer) strategy?

Solution to 1: generated functions!

parse -> expressions -> macro expansion -> new expression -> type inference -> generated functions -> compile -> run

[TensorOperations.jl kernels](https://github.com/Jutho/TensorOperations.jl/blob/master/src/implementation/kernels.jl)

Solution to 2: divide dimensions along which the minimum of the memory jumps of the two arrays is maximal.

### 2. Partial trace
* very similar, but somewhat more carefull

### 3. Tensor contraction: very similar to matrix multiplication

* Fastest algorithm: permute input arrays and reshape them such that you can use BLAS matrix multiplication
  <img src="simplecontraction.png" style="width: 400px;"/>

```julia
Amat=reshape(permutedims(A,[1,4,2,3]),(dA1*dA4,dA2*dA3))
Bmat=reshape(permutedims(B,[3,1,2]),(dB3*dB1,dB2))
Cmat=Amat*Bmat
C=permutedims(reshape(Cmat,(dA1,dA4,dB2)),[1,3,2])
```

```julia
using TensorOperations
C = tensorcontract(A,[1,2,3,4],B,[3,5,2],[1,5,4])
@tensor C[a,b,c] = A[a,d,e,c]*B[e,b,d]
```

## Optimization of tensor contraction order

### Contraction order matters!

* matrix - matrix - vector multiplication: `A*B*v`: 
  
  `A*(B*v)` is much faster than `(A*B)*v`


* Optimal contraction order in more complicated tensor networks?
  <img src="mera.png" style="width: 200px;"/>
  
* Pairwise contraction is always sufficient, but in which sequence?

### What is optimal contraction sequence?

* Manual determination can become laborious task
* Contraction of two-dimensional multiscale entanglement renormalization ansatz:
  <img src="2dmerac.png" style="width: 800px;"/>

### Algorithmic determination of optimal contraction sequence

"Faster identification of optimal contraction sequences for tensor networks"

Robert N. C. Pfeifer, JH, and Frank Verstraete, Phys Rev E 90, 033315 (2014)

* Breadth-first constructive approach:
  <img src="algorithm.png" style="width: 500px;"/>
* Add tricks to make it efficient   

  <img src="mera2.png" style="width: 800px;"/>
  

In [21]:
using TensorOperations
x=TensorOperations.Power(1,1);
labels=Any[[-1,1,2],[-2,3,4],[-3,5,6],[2,3,7,8],[4,5,9,10],[7,8,9,11,12,13],
    [14,15,11,12],[16,17,13,10],[-4,1,14],[-5,15,16],[-6,17,6]];
costW = [x,x,x]
costU = [x,x,x,x]
costh = [x,x,x,x,x,x]
costs = Any[costW,costW,costW,costU,costU,costh,costU,costU,costW,costW,costW]
cost,tree=optimizecontract(labels,costs);
@show cost;
@show tree;

cost = 2*x^9 + 4*x^8 + 2*x^6 + 2*x^5
tree = ((3,11),((10,8),((5,2),((9,1),(7,(6,4))))))


In [23]:
x=6;
W1=W2=W3=randn(x,x,x);
U1=U2=randn(x,x,x,x);
h=randn(x,x,x,x,x,x);
@time @tensor begin
    result[-1,-2,-3,-4,-5,-6] := 
        W1[-1,1,2]*W2[-2,3,4]*W3[-3,5,6]*
        U1[2,3,7,8]*U2[4,5,9,10]*
        h[7,8,9,11,12,13]*
        conj(U1)[14,15,11,12]*conj(U2)[16,17,13,10]*
        conj(W1)[-4,1,14]*conj(W2)[-5,15,16]*conj(W3)[-6,17,6];
end
@time @tensoropt χ begin
    result2[-1,-2,-3,-4,-5,-6] := 
        W1[-1,1,2]*W2[-2,3,4]*W3[-3,5,6]*
        U1[2,3,7,8]*U2[4,5,9,10]*
        h[7,8,9,11,12,13]*
        conj(U1)[14,15,11,12]*conj(U2)[16,17,13,10]*
        conj(W1)[-4,1,14]*conj(W2)[-5,15,16]*conj(W3)[-6,17,6]
end;

  0.760331 seconds (1.54 k allocations: 1.094 GB, 37.61% gc time)
  0.006819 seconds (1.32 k allocations: 9.754 MB)


In [24]:
vecnorm(result-result2)/vecnorm(result)

8.856717807695058e-16

## Outlook
Possible features that might be added to `TensorOperations.jl` in the future:
* More flexible index notation
    * to allow for a mixed combination with manual loops in order to take slices
    * to automatically specify reshapes: e.g. `C[(a,b),c,d] = A[a,c,e]*B[b,d,e]`
* Multi-threading support?
    * Currently being implemented in Julia
* GPU support
* Combine directly with BLAS kernels
* ...

## Conclusions
* Julia is a promising language for developing tensor algorithms (and much more)